In [21]:
import numpy as np
import scipy.sparse as sp
import pandas as pd

import sys
sys.path.append('..')
import src

In [22]:
PARAMS = {
    'community': 'southpark',
    'experiment_name': 'status_quo',
    # resource types should be the same as the column names in the resource_preference_pmf and resource_inventory_pmf
    'resource_types': ['water', 'food', 'meds', 'transp', 'comm',
                       'faid', 'warm', 'sani', 'power', 'shelter'], 
    'tau': 4, # isolation days
    'num_scenarios': 100,
    'n': 0.82661451, # degree distribution parameter
    'p': 0.06686762, # degree distribution parameter
    'social_tie': {
        'types': ['strong', 'weak'],
        'pmf':  [0.298, 0.702]
    },
    'distance_decay_alpha': -1.43, # distance decay parameter
    'distances_mat': np.load('../temp/southpark_distance_matrix.npy'), # distance matrix
    'sharing_preference_pmf': pd.read_csv('../temp/southpark_share_pref_PMF.csv'), # sharing preference matrix
    'sharing_priority': [3, 2, 1], # sharing priorities to [strong tie, weak tie, stranger]
    'resource_inventory_pmf': pd.read_csv('../temp/southpark_resource_distr_PMF.csv') # resource inventory distribution
}

In [23]:
def p2p_single_resource_sharing(resource:str, 
                                tau:float, # isolation days
                                n:float, # negative binomial distribution parameter
                                p:float, # negative binomial distribution parameter
                                distance_decay_alpha:float, # distance decay parameter
                                distance_mat:np.matrix, # distance matrix
                                sharing_preference_PMF:pd.Series, 
                                resource_inventory_PMF:pd.Series,
                                social_tie_pmf:list,
                                social_tie_types:list = ['strong', 'weak'],
                                sharing_priority:list = [3, 2, 1],
                                seed:int=0):
    """Step 1: Community-based social network construction"""
    # create a community
    community = src.Community()
    num_nodes = distance_mat.shape[0]
    community.add_nodes_from(range(num_nodes))

    # generate social ties
    np.random.seed(seed)
    degree_list = np.random.negative_binomial(n, 
                                              p, 
                                              size=num_nodes)
    community.generate_social_ties(degrees=degree_list, 
                                   distance_matrix=distance_mat, 
                                   distance_decay_alpha=distance_decay_alpha, 
                                   seed=seed+1)
    community.split_social_ties(types=social_tie_types, 
                                pmf=social_tie_pmf, 
                                seed=seed+2)

    """Step 2: Community-based resource-sharing network construction"""
    # generate sharing preferences
    community.generate_sharing_preference(recipient=[0, 1, 2, 3], # [nobody, strong tie only, strong and weak ties only, everybody]
                                          pmf=sharing_preference_PMF, 
                                          name=resource+'_share_preference',
                                          seed=seed+3)

    """Step 3: Community-based resource inventory"""
    community.generate_resource_inventory(support=np.arange(0, len(resource_inventory_PMF)), 
                                          pmf=resource_inventory_PMF, 
                                          name=resource, 
                                          seed=seed+4)

    # create sharing network
    share_network = community.get_share_network(sharing_preference=resource+'_share_preference', 
                                                priority=sharing_priority)

    """Step 4: Community-based P2P resource-sharing model"""
    priority_matrix = community.get_share_priority_matrix(sharing_preference=resource+'_share_preference', 
                                                          priority=sharing_priority)
    resource_inventory = [community.nodes[n][resource] for n in community.nodes]
    model = src.ResourceSharingModel(wts_matrix=priority_matrix, 
                                     inventory=resource_inventory, 
                                     isolation_days=tau)
    model.solve()

    return community, share_network, model

In [24]:
# main script for running the status quo model
for sce in range(PARAMS['num_scenarios']):
    
    # directory name for saving the results
    dir = PARAMS['community'] + '/' + PARAMS['experiment_name'] + '/tau_' + str(PARAMS['tau']) # directory name
    # Create a dataframe to store the results
    inventory_df = pd.DataFrame(columns=PARAMS['resource_types'])
    resource_df = pd.DataFrame(columns=PARAMS['resource_types'])
    
    # run the model for each resource type
    for rsc in PARAMS['resource_types']:
        community, share_network, model = p2p_single_resource_sharing(resource=rsc,
                                                                    tau=PARAMS['tau'],
                                                                    n=PARAMS['n'],
                                                                    p=PARAMS['p'],
                                                                    social_tie_types = PARAMS['social_tie']['types'],
                                                                    social_tie_pmf = PARAMS['social_tie']['pmf'],
                                                                    distance_decay_alpha = PARAMS['distance_decay_alpha'],
                                                                    distance_mat = PARAMS['distances_mat'],
                                                                    sharing_preference_PMF = PARAMS['sharing_preference_pmf'][rsc],
                                                                    sharing_priority = PARAMS['sharing_priority'],
                                                                    resource_inventory_PMF = PARAMS['resource_inventory_pmf'][rsc],
                                                                    seed=sce)
        inventory_df[rsc] = model.inventory.flatten()
        resource_df[rsc] = model.resource.flatten()

        # save the sharing priority matrix in a sparse matrix
        sp.save_npz(dir + '/wij_mat_sce_{}_{}.npz'.format(sce, rsc), sp.csr_matrix(model.wts_matrix))
        # save the sharing plan in a sparse matrix
        sp.save_npz(dir + '/xij_mat_sce_{}_{}.npz'.format(sce, rsc), sp.csr_matrix(model.share_plan))

    # save the resource inventory vector in csv file
    inventory_df.to_csv(dir + '/q_vec_sce_{}.csv'.format(sce), index=False)
    # save the final resource vector in csv file
    resource_df.to_csv(dir + '/h_vec_sce_{}.csv'.format(sce), index=False)

/Users/lzy95/Documents/GitHub/Peer-to-peer-community-sharing/results/../src/community.py:230: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self).todense()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2758 rows, 1901641 columns and 3803282 nonzeros
Model fingerprint: 0xfcc44303
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 1751 rows and 1788794 columns
Presolve time: 0.40s
Presolved: 1007 rows, 112847 columns, 225647 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.043e+04
 Factor NZ  : 1.076e+05 (roughly 25 MB of memory)
 Factor Ops : 1.857e+07 (less than 1 second per iteration)
 Threads    : 8

Barrier performed 0 iterations in 0.49 seconds (1.04 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with primal simplex

Use c

/Users/lzy95/Documents/GitHub/Peer-to-peer-community-sharing/results/../src/community.py:230: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self).todense()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2758 rows, 1901641 columns and 3803282 nonzeros
Model fingerprint: 0x69408024
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2050 rows and 1842499 columns
Presolve time: 0.44s
Presolved: 708 rows, 59142 columns, 118205 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.270e+04
 Factor NZ  : 4.416e+04 (roughly 10 MB of memory)
 Factor Ops : 7.494e+06 (less than 1 second per iteration)
 Threads    : 8

Barrier performed 0 iterations in 0.48 seconds (1.14 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with primal simplex
Iteratio

/Users/lzy95/Documents/GitHub/Peer-to-peer-community-sharing/results/../src/community.py:230: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self).todense()


...

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2758 rows, 1901641 columns and 3803282 nonzeros
Model fingerprint: 0x1907686e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2148 rows and 1858473 columns
Presolve time: 0.63s
Presolved: 610 rows, 43168 columns, 86252 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.434e+04
 Factor NZ  : 2.600e+04 (roughly 6 MB of memory)
 Factor Ops : 2.950e+06 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.68 seconds (0.99 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration   

/Users/lzy95/Documents/GitHub/Peer-to-peer-community-sharing/results/../src/community.py:230: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self).todense()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2758 rows, 1901641 columns and 3803282 nonzeros
Model fingerprint: 0xe4b95e42
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 1735 rows and 1775771 columns
Presolve time: 0.51s
Presolved: 1023 rows, 125870 columns, 251691 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.088e+04
 Factor NZ  : 1.226e+05 (roughly 30 MB of memory)
 Factor Ops : 2.315e+07 (less than 1 second per iteration)
 Threads    : 8

Barrier performed 0 iterations in 0.66 seconds (1.08 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with primal simplex
Iterat

/Users/lzy95/Documents/GitHub/Peer-to-peer-community-sharing/results/../src/community.py:230: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self).todense()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2758 rows, 1901641 columns and 3803282 nonzeros
Model fingerprint: 0xaddff753
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2049 rows and 1845852 columns
Presolve time: 0.50s
Presolved: 709 rows, 55789 columns, 111489 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.462e+04
 Factor NZ  : 4.750e+04 (roughly 10 MB of memory)
 Factor Ops : 7.977e+06 (less than 1 second per iteration)
 Threads    : 8

Barrier performed 0 iterations in 0.56 seconds (1.02 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with primal simplex
Iteratio